In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [92]:
price = pd.read_csv('data/BrandAverageRetailPrice.csv')
details = pd.read_csv('data/BrandDetails.csv')
sales = pd.read_csv('data/BrandTotalSales.csv')
units = pd.read_csv('data/BrandTotalUnits.csv')

# storing all of the dataframes in a dictionary for easier use
dfs = {
    'price': price,
    'details': details,
    'sales': sales,
    'units': units,
}

In [96]:
price.columns, price.shape

(Index(['Brands', 'Months', 'ARP', 'vs. Prior Period'], dtype='object'),
 (27211, 4))

In [67]:
details.columns, details.shape

(Index(['State', 'Channel', 'Category L1', 'Category L2', 'Category L3',
        'Category L4', 'Category L5', 'Brand', 'Product Description',
        'Total Sales ($)', 'Total Units', 'ARP', 'Flavor', 'Items Per Pack',
        'Item Weight', 'Total THC', 'Total CBD', 'Contains CBD', 'Pax Filter',
        'Strain', 'Is Flavored', 'Mood Effect', 'Generic Vendor',
        'Generic Items', '$5 Price Increment'],
       dtype='object'),
 (144977, 25))

In [68]:
sales.columns, sales.shape

(Index(['Months', 'Brand', 'Total Sales ($)'], dtype='object'), (25279, 3))

In [69]:
units.columns, units.shape

(Index(['Brands', 'Months', 'Total Units', 'vs. Prior Period'], dtype='object'),
 (27686, 4))

In [70]:
price.describe(include='all')

,Brands,Months,ARP,vs. Prior Period
count,27211,27211,25279.000000,24499.000000
unique,1627,37,NaN,NaN
top,Farmaceuticals Company,06/2021,NaN,NaN
freq,37,909,NaN,NaN
mean,NaN,NaN,22.679732,-0.065028
std,NaN,NaN,19.802724,0.388923
min,NaN,NaN,0.000000,-1.000000
25%,NaN,NaN,10.512827,-0.088073
50%,NaN,NaN,17.033051,-0.011649
75%,NaN,NaN,31.505612,0.045232


In [71]:
details.describe(include='all')

,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
count,144977,144977,144977,144977,144245,102618,50135,144977,144977,144977,...,144977,144977,144977,44301,115639,11287,144977,144977,144977,144977
unique,1,1,5,13,53,71,38,1123,32608,133144,...,136,149,2,2,5824,2,2,2,2,22
top,California,Licensed,Inhaleables,Concentrates,Vape,Vape Cartridge,Live Resin Cartridge,Raw Garden,"Pre Rolled - Strain Blends - Pre-Rolled, 1.00g...",168.17203,...,0,0,THC Only,Not Pax,Hybrid Strain Blends,Flavored,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$25.00 to $29.99
freq,144977,144977,121859,83534,44301,38488,19962,9004,790,136,...,130154,138362,134332,42135,4936,7807,137990,137370,138667,13832
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
sales.describe(include='all')

,Months,Brand,Total Sales ($)
count,25279,25279,25279
unique,37,1627,25277
top,05/2021,Quim Rock,0
freq,848,37,3


In [73]:
units.describe(include='all')

,Brands,Months,Total Units,vs. Prior Period
count,27686,27686,25712,24935.000000
unique,1640,38,24494,NaN
top,Dr. Raw Organics,06/2021,268.45435100000000,NaN
freq,38,909,6,NaN
mean,NaN,NaN,NaN,0.265306
std,NaN,NaN,NaN,3.291373
min,NaN,NaN,NaN,-1.000000
25%,NaN,NaN,NaN,-0.351822
50%,NaN,NaN,NaN,-0.055216
75%,NaN,NaN,NaN,0.240113


## 1. Merging datasets
Merging datasets and linking information into a single dataframe for analysis.

Since `details` was the only dataframe with any columns with a single unique value, we can drop all of these columns since they are redundant.

In [79]:
def drop_identical_data(df):
    '''
    Drop columns that contain only 1 unique value.
    
    Args:
        df: Dataframe
    Returns:
        Dataframe with columns that have 2 or more unique
        values each.
    '''
    for column in df.columns:
        if len(df[column].unique()) == 1:
            df = df.drop(column, axis=1)
            print(f'Dropped {column}')
    return df

for i in dfs:
    dfs[i] = drop_identical_data(dfs[i])

For consistency, renaming `Brands` to `Brand` and placing `Brand` column at the start of every dataframe.

In [108]:
def move_to_front(label, df):
    '''
    Moves column `label` to the front of the dataframe. If label
    doesn't exist, it will return `df` untouched.
    
    Args:
        column: str
        df: Dataframe
    Returns:
        Dataframe with `label` in the front (index 0)
    '''
    if label not in df.columns: return df
    column = df[label]
    df = df.drop(label, axis=1)
    df.insert(0, label, column)
    return df

# renaming brands to brand
for i in dfs:
    if 'Brands' in dfs[i].columns:
        dfs[i] = dfs[i].rename(columns={'Brands': 'Brand'})
        
# reording columns
for i in dfs:
    dfs[i] = move_to_front('Brand', dfs[i])


In [112]:
for i in dfs:
    print(dfs[i].columns)

Index(['Brand', 'Months', 'ARP', 'vs. Prior Period'], dtype='object')
Index(['Brand', 'Category L1', 'Category L2', 'Category L3', 'Category L4',
       'Category L5', 'Product Description', 'Total Sales ($)', 'Total Units',
       'ARP', 'Flavor', 'Items Per Pack', 'Item Weight', 'Total THC',
       'Total CBD', 'Contains CBD', 'Pax Filter', 'Strain', 'Is Flavored',
       'Mood Effect', 'Generic Vendor', 'Generic Items', '$5 Price Increment'],
      dtype='object')
Index(['Brand', 'Months', 'Total Sales ($)'], dtype='object')
Index(['Brand', 'Months', 'Total Units', 'vs. Prior Period'], dtype='object')


## 2. Time Series Extraction
Developing timeseries features to augment the dataset and to enable timeseries predictive models. 

# data[column] = pd.to_datetime(data[column], format='%m/%Y')